參考: https://ppfocus.com/0/ed5ea2fb3.html

In [48]:
from sklearn import datasets
from sklearn import svm
import numpy as np
import pandas as pd

iris = datasets.load_iris() 
print(type(iris), dir(iris))

x = iris.get('data')
y = iris.get('target')


<class 'sklearn.utils.Bunch'> ['DESCR', 'data', 'data_module', 'feature_names', 'filename', 'frame', 'target', 'target_names']


In [49]:
# 隨機劃分訓練集和測試集
num = x.shape[0] # 樣本總數
ratio = 7/3 # 劃分比例，訓練集數目:測試集數目
num_test = int(num/(1+ratio)) # 測試集樣本數目
num_train = num -  num_test # 訓練集樣本數目
index = np.arange(num) # 產生樣本標號
np.random.shuffle(index) # 洗牌
x_test = x[index[:num_test],:] # 取出洗牌後前 num_test 作為測試集
y_test = y[index[:num_test]]
x_train = x[index[num_test:],:] # 剩餘作為訓練集
y_train = y[index[num_test:]]


#一對一法（one-versus-one，OVO SVMs）

In [50]:
clf = svm.SVC(decision_function_shape="ovo")
clf.fit(x_train, y_train)
y_predict = clf.predict(x_test)

In [51]:
# 計算分類準確率
acc = sum(y_predict==y_test)/num_test
print('The accuracy is', acc) 

The accuracy is 0.9777777777777777


In [52]:
clf_linear = svm.SVC(decision_function_shape="ovo", kernel="linear")
clf_rbf = svm.SVC(decision_function_shape="ovo", kernel="rbf")
clf_linear.fit(x_train, y_train)
clf_rbf.fit(x_train, y_train)

y_test_pre_linear = clf_linear.predict(x_test)
y_test_pre_rbf = clf_rbf.predict(x_test)


In [53]:
# 計算分類準確率
acc_linear = sum(y_test_pre_linear==y_test)/num_test
print('linear kernel: The accuracy is', acc_linear) 
acc_rbf = sum(y_test_pre_rbf==y_test)/num_test
print('rbf kernel: The accuracy is', acc_rbf) 

linear kernel: The accuracy is 0.9777777777777777
rbf kernel: The accuracy is 0.9777777777777777


#一對多法（one-versus-rest，OVR SVMs）

In [54]:
clf = svm.SVC(decision_function_shape="ovr")
clf.fit(x_train, y_train)
y_predict = clf.predict(x_test)

In [55]:
# 計算分類準確率
acc = sum(y_predict==y_test)/num_test
print('The accuracy is', acc) 

The accuracy is 0.9777777777777777


In [56]:
clf_linear2 = svm.SVC(decision_function_shape="ovr", kernel="linear")
clf_rbf2 = svm.SVC(decision_function_shape="ovr", kernel="rbf")
clf_linear2.fit(x_train, y_train)
clf_rbf2.fit(x_train, y_train)

y_test_pre_linear2 = clf_linear2.predict(x_test)
y_test_pre_rbf2 = clf_rbf2.predict(x_test)

In [57]:
# 計算分類準確率
acc_linear2 = sum(y_test_pre_linear2==y_test)/num_test
print('linear kernel: The accuracy is', acc_linear2) 
acc_rbf2 = sum(y_test_pre_rbf2==y_test)/num_test
print('rbf kernel: The accuracy is', acc_rbf2) 

linear kernel: The accuracy is 0.9777777777777777
rbf kernel: The accuracy is 0.9777777777777777


#進行SVC 優化

主要調節的參數有：C、kernel、degree、gamma、coef0，其中：

C: 懲罰係數，越大，趨向於對訓練集全分對的情況，這樣對訓練集測試時準確率很高，但泛化能力弱；C越小，對誤分類的懲罰減小，允許容錯，將他們當成噪聲點，泛化能力較強。

kernel：核函數，默認是rbf，可以是『linear』, 『poly』, 『rbf』, 『sigmoid』, 『precomputed』 

gamma 越大，支持向量越少，gamma 越小，支持向量越多。支持向量的個數影響訓練和預測的速度。

degree: 多項式的次數越高，準確率就越高

In [58]:
# 比較不同kernel
from sklearn.metrics import recall_score

def svc(kernel):
  return svm.SVC(kernel=kernel,decision_function_shape="ovo")

def nusvc():
  return svm.NuSVC(decision_function_shape="ovo")

def linearsvc():
  return svm.LinearSVC(multi_class="ovr")

def modelist():
  modelist = []
  kernalist = {"linear", "poly", "rbf", "sigmoid"}
  for each in kernalist:
    modelist.append(svc(each))
  modelist.append(nusvc())
  modelist.append(linearsvc())
  return modelist

def svc_model(model):
  model.fit(x_train, y_train)
  acu_train = model.score(x_train, y_train)
  acu_test = model.score(x_test, y_test)
  y_pred = model.predict(x_test)
  recall = recall_score(y_test, y_pred, average="macro")
  return acu_train, acu_test, recall

def run_svc_model(modelist):
  result = {"kernel":[],"acu_train":[],"acu_test":[],"recall":[]}
  for model in modelist:
    acu_train,acu_test,recall = svc_model(model)
    try:
      result["kernel"].append(model.kernel)
    except:
      result["kernel"].append(None)
    result["acu_train"].append(acu_train)
    result["acu_test"].append(acu_test)
    result["recall"].append(recall)

  return pd.DataFrame(result)

run_svc_model(modelist()) # kernel=linear的accuracy跟recall最好

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


,kernel,acu_train,acu_test,recall
0,linear,0.990476,0.977778,0.980392
1,rbf,0.971429,0.977778,0.980392
2,sigmoid,0.371429,0.244444,0.333333
3,poly,0.980952,0.977778,0.980392
4,rbf,0.971429,0.977778,0.980392
5,None,0.971429,0.911111,0.921569


In [59]:
# 我們發現kernel=linear表現最好，因此以下用linear
def test_c():
  result = {"C":[],"acu_train":[],"acu_test":[],"recall":[]}
  for c in range(1,50,5):
    model = svm.SVC(kernel="linear",C=c,decision_function_shape="ovo")
    acu_train,acu_test,recall = svc_model(model)
    result["C"].append(c)
    result["acu_train"].append(acu_train)
    result["acu_test"].append(acu_test)
    result["recall"].append(recall)

  df = pd.DataFrame(result)
  return df

test_c() # C=1的accuracy跟recall最好

,C,acu_train,acu_test,recall
0,1,0.990476,0.977778,0.980392
1,6,0.990476,0.955556,0.960784
2,11,0.990476,0.955556,0.960784
3,16,0.990476,0.955556,0.960784
4,21,0.980952,0.955556,0.960784
5,26,0.980952,0.955556,0.960784
6,31,0.980952,0.955556,0.960784
7,36,0.980952,0.955556,0.960784
8,41,0.980952,0.955556,0.960784
9,46,0.980952,0.955556,0.960784


In [60]:
# 我們發現kernel=linear表現最好，因此以下用linear
def test_gamma():
  result = {"gamma":[],"acu_train":[],"acu_test":[],"recall":[]}
  for g in range(1,50,5):
    model = svm.SVC(kernel="linear",gamma=g,decision_function_shape="ovo")
    acu_train,acu_test,recall = svc_model(model)
    result["gamma"].append(g)
    result["acu_train"].append(acu_train)
    result["acu_test"].append(acu_test)
    result["recall"].append(recall)

  df = pd.DataFrame(result)
  return df

test_gamma() # gamma值不影響

,gamma,acu_train,acu_test,recall
0,1,0.990476,0.977778,0.980392
1,6,0.990476,0.977778,0.980392
2,11,0.990476,0.977778,0.980392
3,16,0.990476,0.977778,0.980392
4,21,0.990476,0.977778,0.980392
5,26,0.990476,0.977778,0.980392
6,31,0.990476,0.977778,0.980392
7,36,0.990476,0.977778,0.980392
8,41,0.990476,0.977778,0.980392
9,46,0.990476,0.977778,0.980392


In [62]:
# 我們發現kernel=linear表現最好，因此以下用linear
def test_degree():
  result = {"degree":[],"acu_train":[],"acu_test":[],"recall":[]}
  for d in range(1,11,1):
    model = svm.SVC(kernel="linear",degree=d,decision_function_shape="ovo")
    acu_train,acu_test,recall = svc_model(model)
    result["degree"].append(d)
    result["acu_train"].append(acu_train)
    result["acu_test"].append(acu_test)
    result["recall"].append(recall)

  df = pd.DataFrame(result)
  return df

test_degree() # degree值不影響

,degree,acu_train,acu_test,recall
0,1,0.990476,0.977778,0.980392
1,2,0.990476,0.977778,0.980392
2,3,0.990476,0.977778,0.980392
3,4,0.990476,0.977778,0.980392
4,5,0.990476,0.977778,0.980392
5,6,0.990476,0.977778,0.980392
6,7,0.990476,0.977778,0.980392
7,8,0.990476,0.977778,0.980392
8,9,0.990476,0.977778,0.980392
9,10,0.990476,0.977778,0.980392
